# Intensifying Shapes for a TSA

## Overview
### System Layout and Overall Goal
The overall system dimensions of an activated carbon adsorbent layer are shown in the image below, and the aim is to choose a shape that intensifies the TSA operation, when the Pressure Drop $\Delta P_D = 20 \frac{kPa}{m*{2}} $. This Notebook examines 6 simple shape archetypes, as a preliminary to a more advanced study.


![Adsorbent Layer](./images/intensify/adsorbent_layer.png)

It can be seen that the adsorbent layer has thin copper layers (orange), $ 200 \mu m $, over which hot or cold water can flow. Internally, the adsorbent shape (dark grey) is undefined, and the goal is to produce the most adsorbate volume (e.g. CO2 or CH4) per unit-volume of Adsorber System, using the least paid-for energy possible. Assuming the system uses the existing pressure drop in the flow stream (i.e. $20 \frac{20 kPa}{m^{2}}$), and the temperature swing is driven by waste energy (i.e $T_{a} = 300 K, T_{d} = 368 K$), then the paid-for-energy amounts to pumping energy to circulate the water.

### Critical Trade-Offs for each Shape Instance
But the adsorbate volume goal is comprised of trade-offs between multiple sub-goals, starting with 3 orthognal decisions:
1. Volume Adsorbed per Cycle vs Layer Volume
2. Time to Adsorb vs Time to Swing
3. Quality Returned, Chromotographic Graph vs Square Graph

It is expected that each Shape Candidate is likely to have a unique series of performance metrics across these 3 sub goals.

### 3 Shape Families, 2 Shape Candidates from Each
The initial examination is over 3 shape families that can be analysed analytically, with two variations on each that meet $\Delta P_D$ hard limit, in total 6 Shape Candidates:
1. Spheres: Close-Packed vs Square-Packed
2. Monolith: Few Big Tubes vs Many Small Tubes 
3. Laminates: Thick Laminates vs Thin Laminates

### Single Adsorbent Material, Across a Range of Concentrations
All Shapes are constructed with the same Activated Carbon material, in this case Maxsorb, so the focus can be solely on shape performance, and how it may vary with concentration (i.e. application). 

The adsorption performance of each Shape Candidate will then be examined across a range of adsorbate concentrations listed in the table below, [(CO2 Source)](https://pubs.acs.org/doi/10.1021/acs.energyfuels.1c01618).

Type | CH4 Molar Fraction | Partial Pressure (kPa)
:-----------|:------------:|------------:
Water-Gas Shift Concentration | $$\gamma_H = 60\% $$ | $$p_{A_H} = P_{sys} \cdot  \gamma_H = 1.336 kPa$$
Post-combustion Pulvarised Coal Concentration | $$\gamma_M = 15\%$$ | $$p_{A_H} = P_{sys} \cdot  \gamma_H = 0.89 kPa$$
Post-combustion Gas Concentration | $$\gamma_M = 4\%$$ | $$p_{A_H} = P_{sys} \cdot  \gamma_H = 0.89 kPa$$
VAM High Concentration | $$\gamma_P = 0.5\%$$ | $$p_{A_H} = P_{sys} \cdot  \gamma_H = 0.557 kPa$$
VAM Low Concentration | $$\gamma_T = 0.2\%$$ | $$p_{A_H} = P_{sys} \cdot  \gamma_H = 0.223 kPa$$
Direct Air Capture (DAC) Concentration | $$\gamma_{CO_2} = \frac{400}{1,000,000}$$ | $$p_{A_H} = P_{sys} \cdot  \gamma_H = 0.045 kPa$$

### Simplest Possible Analysis is Applied
We will use analytical relations to define the shapes based on pressure drop. The thermal and adsorption performance of each Shape Candidate is modelled using simple 1D/2-D difference calculations,across each of the concentrations. The results are then compared.

### System Parameters

In [5]:
System = {
    "w": 1000e-3,     # System width
    "l": 500e-3,      # System length
    "h": 20e-3,       # System height
    "h_g": 105e-6,    # Laminate air gap height
    "h_t": 100e-6,    # Laminate thickness
    "P_sys": 101.325e3,# Air Pressure
    "PD_per_m": 20,# Pressure dropm objective kPa per metre
    "T_a": 300,    # Temeperature a - Adsorption
    "T_d": 368,     # Temperature d - Desorption
    "rho_a": 1.17, # denisty in kg/m3
    "mu_a": 1.846e-5, # dynamic viscosity 
    "M_CH4": 16.043, # gm/mol
    "U_sys": 1, # system velocity    
    "R": 8.314462, # Universal Gas Constant J K−1 mol−1
    
    
}
Gamma = {
    "WGS": 0.60, # Water-Gas Shift CO2 Concentration, 60% CO2
    "PC": 0.15, # Pulvarised Coal CO2 Concentration, 15% CO2
    "NGCC": 0.04, # Natural Gas CO2 Concentration, 4% CO2
    "VAMH": 0.008, # High VAM Ventilation Air Methane Concentration, 8% CH4
    "VAML": 0.003, # Low VAM Ventilation Air Methane Concentration, 3% CH4
    "DAC": 0.0004 # Air-CO2 Ambient Concentration, 400 ppm CO2
}
Maxsorb = {
    "C_m": 0.407, # Toth monolayer adsorption capacity
    "k_o":9.35e-4, # Toth pre-exponential constant
    "delta_h": 1867, # isoteric heat of adsorption, divide by R
    "t": 0.67, # dimensionless Toth constant
    "rho_m": 680, # kg/m3 maxsorb density
    "k_Lch4": 2.947, #mass transfer of ch4
    "k_Ln2": 8.255, # mass transfer of n2
    "D_eff": 7.4e-8 #m2/s effective diffusivity
}
Conductivity = {
    "k_ac": 0.63, # (W/m2K) activated carbon thermal conductivity
    "cp_ac": 900, # (J/kg.K)
    "k_cf": 100, # (W/m2K) carbon fibre thermal conductivity
    "cp_cf": 800, # (J/kg.K)
    "rho_cf": 1750, # (kg/m3)
    "k_cu": 401,  # (W/m2K) copper thermal conductivity, wikipedia
    "cp_cu": 385, # (J/kg.K)
    "rho_cu": 8960, # (kg/m3)
    "k_h20" : 0.6 # W/(m2.K) water thermal conductivity, wikipedia
}
Model = {
    "num_slices" : 10, # the number of discretised slices
    "num_length" : 10000, # the number of discrete blocks in 500 mm length
    "dic_width" : 0.5e-5, # the width of the discrete block
    "time_delta" : 0.0001 # the time step to use for the model
}